## import modules

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

## tokenizer
> convert the character to and from integers

In [2]:
with open('history.txt', 'r', encoding='utf-8') as f:
    text = f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i,c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

check out the first 100 characters

In [3]:
text[:100]

'\u3000\u3000卷一·五帝本纪第一\n\n\u3000\u3000黄帝者，少典之子，姓公孙，名曰轩辕。生而神灵，弱而能言，幼而徇齐，长而敦敏，成而聪明。轩辕之时，神农氏世衰。诸侯相侵伐，暴虐百姓，而神农氏弗能征。于是轩辕乃习用干戈，以征'

In [4]:
len(vocab)

4774

In [5]:
len(text)

644266

words frequency

In [6]:
len(text)/len(vocab)

134.95307917888562

encoded integers

In [7]:
encoded[:100]

array([  18,   18,  506,   30,    7,   98, 1137, 1776, 3008, 2879,   30,
          0,    0,   18,   18, 4695, 1137, 3153, 4770, 1021,  333,   70,
        937, 4770,  874,  323,  942, 4770,  566, 1750, 3965, 3995,   20,
       2545, 3155, 2756, 2331, 4770, 1218, 3155, 3226, 3669, 4770, 1153,
       3155, 1249, 4721, 4770, 4280, 3155, 1634, 1623, 4770, 1437, 3155,
       3175, 1702,   20, 3965, 3995,   70, 1692, 4770, 2756,  346, 2034,
         45, 3590,   20, 3755,  219, 2646,  220,  156, 4770, 1745, 3491,
       2615,  874, 4770, 3155, 2756,  346, 2034, 1206, 3226, 1245,   20,
         94, 1714, 3965, 3995,   67,   83, 2547, 1147, 1431, 4770,  143,
       1245], dtype=int32)

In [8]:
int_to_vocab

{0: '\n',
 1: "'",
 2: '.',
 3: '`',
 4: 'k',
 5: '{',
 6: '}',
 7: '·',
 8: 'Я',
 9: '‘',
 10: '’',
 11: '“',
 12: '”',
 13: '…',
 14: '┱',
 15: '■',
 16: '□',
 17: '◎',
 18: '\u3000',
 19: '、',
 20: '。',
 21: '々',
 22: '《',
 23: '》',
 24: '䌷',
 25: '䎬',
 26: '䜣',
 27: '䲡',
 28: '䴔',
 29: '䴖',
 30: '一',
 31: '丁',
 32: '七',
 33: '万',
 34: '丈',
 35: '三',
 36: '上',
 37: '下',
 38: '不',
 39: '与',
 40: '丐',
 41: '丑',
 42: '专',
 43: '且',
 44: '丕',
 45: '世',
 46: '丘',
 47: '丙',
 48: '业',
 49: '丛',
 50: '东',
 51: '丝',
 52: '丞',
 53: '两',
 54: '严',
 55: '丧',
 56: '个',
 57: '中',
 58: '丰',
 59: '临',
 60: '丸',
 61: '丹',
 62: '为',
 63: '主',
 64: '丽',
 65: '举',
 66: '乂',
 67: '乃',
 68: '久',
 69: '义',
 70: '之',
 71: '乌',
 72: '乍',
 73: '乎',
 74: '乏',
 75: '乐',
 76: '乔',
 77: '乖',
 78: '乘',
 79: '乙',
 80: '九',
 81: '乞',
 82: '也',
 83: '习',
 84: '乡',
 85: '书',
 86: '买',
 87: '乱',
 88: '乳',
 89: '乾',
 90: '予',
 91: '争',
 92: '事',
 93: '二',
 94: '于',
 95: '亏',
 96: '云',
 97: '互',
 98: '五',
 99: '井',
 100

## Making training mini-batchs

In [9]:
def get_batches(arr, batch_size, n_steps):
    '''
    Create a generator that returns batches of size
       batch_size x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    characters_per_batch = batch_size * n_steps
    n_batches = len(arr)//characters_per_batch
    
    arr = arr[:characters_per_batch*n_batches]
    
    arr = np.reshape(arr,[batch_size,-1])
    
    for n in range(0, arr.shape[1], n_steps):
        x = arr[:, n:n+n_steps]
        y_temp = arr[:,n+1:n+n_steps+1]
        
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:, :y_temp.shape[1]] = y_temp
        
        yield x, y

test the function

In [10]:
batchs = get_batches(encoded, 5, 10)
x, y = next(batchs)

In [11]:
print(x)
print(y)

[[  18   18  506   30    7   98 1137 1776 3008 2879]
 [   0   18   18   98 2002 1707 4770  376 1988   50]
 [4326   70  461 4770  143   50   59 3707 4770  487]
 [3155 3042 1681 1119   70 2043 4770  331  442 1269]
 [1626  133 2945 4770 4331 1050 2089   70 4350  143]]
[[  18  506   30    7   98 1137 1776 3008 2879   30]
 [  18   18   98 2002 1707 4770  376 1988   50   50]
 [  70  461 4770  143   50   59 3707 4770  487  708]
 [3042 1681 1119   70 2043 4770  331  442 1269 1681]
 [ 133 2945 4770 4331 1050 2089   70 4350  143  142]]


## Building the model

### inputs

In [12]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    inputs = tf.placeholder(tf.int32, shape=[batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, shape=[batch_size, num_steps], name= 'targets')
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### Lstm cell

In [13]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    init_state = cell.zero_state(batch_size, dtype=tf.float32)
    
    return cell, init_state

### RNN Output

In [14]:
def build_output(lstm_output, in_size, out_size):
    '''
        Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    '''
    
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal([in_size,out_size],stddev=1.0))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

## Training loss

In [15]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    y_one_hot = tf.one_hot(targets,num_classes)
    y_shaped = tf.reshape(y_one_hot, [-1, num_classes])
    
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_shaped, logits=logits)
    loss = tf.reduce_mean(loss, name='loss')
    
    return loss

### Optimizer

In [16]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

In [17]:
class CharRNN:
    def __init__(self, num_classes, batch_size=64, num_steps=50,
                lstm_size=128, num_layers=2, learning_rate=0.001,
                grad_clip=5, sampling=False):
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps
        
        tf.reset_default_graph()
        
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)
        
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)
        
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot,initial_state=self.initial_state)
        self.final_state = state
        
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

### Hyperparameters

In [25]:
batch_size = 20         # Sequences per batch
num_steps = 50          # Number of sequence steps per batch
lstm_size = 256         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.01    # Learning rate
keep_prob = 0.5         # Dropout keep probability

### Training

In [33]:
epochs = 17
# Print losses every N interations
print_every_n = 50

# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    checkpoint = tf.train.latest_checkpoint('west_checkpoints')
    saver.restore(sess, checkpoint)
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "west_checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "west_checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/17...  Training Step: 50...  Training loss: 4.6356...  0.1743 sec/batch
Epoch: 1/17...  Training Step: 100...  Training loss: 4.5091...  0.1754 sec/batch
Epoch: 1/17...  Training Step: 150...  Training loss: 4.5569...  0.1753 sec/batch
Epoch: 1/17...  Training Step: 200...  Training loss: 4.5951...  0.1754 sec/batch
Epoch: 1/17...  Training Step: 250...  Training loss: 4.5013...  0.1756 sec/batch
Epoch: 1/17...  Training Step: 300...  Training loss: 4.4829...  0.1759 sec/batch
Epoch: 1/17...  Training Step: 350...  Training loss: 4.4728...  0.1760 sec/batch
Epoch: 1/17...  Training Step: 400...  Training loss: 4.6090...  0.1737 sec/batch
Epoch: 1/17...  Training Step: 450...  Training loss: 4.4806...  0.1755 sec/batch
Epoch: 1/17...  Training Step: 500...  Training loss: 4.4709...  0.1762 sec/batch
Epoch: 1/17...  Training Step: 550...  Training loss: 4.4115...  0.1763 sec/batch
Epoch: 1/17...  Training Step: 600...  Training loss: 4.3232...  0.1738 sec/batch
Epoch: 2/17...  T

### Saved checkpoints

In [34]:
tf.train.get_checkpoint_state('west_checkpoints')

model_checkpoint_path: "west_checkpoints/i10948_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i200_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i400_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i600_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i800_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i1000_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i1200_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i1400_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i1600_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i1800_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i2000_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i2200_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i2400_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i2600_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i2800_l256.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i3000_l256.ckpt"


### Sampling

In [28]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [29]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [35]:
tf.train.latest_checkpoint('west_checkpoints')

'west_checkpoints/i10948_l256.ckpt'

In [36]:
checkpoint = tf.train.latest_checkpoint('west_checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="孙")
print(samp)

孙、魏之属。

　　齐王之时，秦人以为郎，以故上书曰：“臣闻大臣不忠，不能以其功，故不敢以故不能得。今君王之为天子之所以为人臣，然不能自为以故为臣，故以为不可。且王之为不能不得于天下之士，以天下为大将，臣不敢以为臣也。且臣之事之所以不敢不得以王之，而以为王也。臣闻之，以故为臣之计，以为臣之事也。臣闻之，以天下之国也。

　　“秦王之所与者，以韩、魏、魏、魏、魏之所以不与魏也，秦以兵攻秦，则韩、魏、魏、赵、魏、韩、魏、赵之地也，不如为秦也。且秦之彊，秦必有兵于秦，韩、魏之兵而彊于秦，则秦攻之而不可。秦以攻魏，魏不能出秦而西地，秦之彊国则秦、赵之地，以秦之兵以西割地以攻韩，韩之兵必可与秦也。秦、魏之攻齐、魏、魏而取秦也，则楚之地不敢与楚也。秦必不听秦也。秦以兵攻楚，则魏以其为大王也。今王不能与秦攻楚，而韩、魏以兵以为楚也。”秦使王翦、韩、魏、赵、魏、韩、魏、韩、魏、魏、魏、魏、魏共伐秦。三十四年，韩、魏、赵、魏、赵、魏、韩、魏、秦、魏、韩、魏、魏、秦、魏、魏、魏共伐魏，取魏阳。十二年，秦伐魏，虏楚将，拔之。

　　十二年，秦拔我三城。二年，秦拔我二年。三十四年，秦拔我阳城。二十一年，与秦攻秦，秦取我中城。十四年，秦取我河东，与魏、魏、赵、韩、魏、魏、赵、魏、魏、赵、韩、魏、魏、魏共共攻魏，拔魏。

　　十一年，与韩、魏会河南，以韩、魏、魏、魏、魏、魏、魏、魏、秦、魏、赵、魏、赵、魏、韩、魏、赵、魏、魏共攻韩、魏，取河、河、魏、魏、魏、韩、魏，取魏之城。四年，秦拔我城阳，虏赵将将。

　　十二年，秦攻魏，取三河。三年，与魏、魏会魏。魏献十二年，与魏攻赵，拔赵。二十四年，齐与赵、赵共击秦，取其城，取赵。十二年，与魏伐秦。十四年，赵复攻魏。秦拔我十二城，取三县，秦兵复出。

　　二十六年，楚败魏于澮。与韩、赵、赵、韩、魏、赵、赵、赵、韩、魏、赵、魏、魏、赵、魏、韩、魏、齐、魏、魏、魏、韩、魏、齐共伐魏，拔之。十五年，与秦会临晋。十六年，秦攻魏，拔之。二十四年，秦伐魏，取我阳城。四年，魏与魏、魏共攻赵，取魏、蒲。十五年，秦伐我。十一年，齐与韩、魏伐魏。十二年，与魏攻魏，取河上，取魏阳。十一年，秦拔我城。十四年，秦攻韩、魏，取之。十二年，秦拔我城。十五年，秦败我军于城。二年，秦围我阳城，取魏地。三十二年，秦拔我城阳。二十六年，秦拔我城，取魏城。二十五年，魏、韩、魏、魏、魏、赵、赵

In [37]:
checkpoint = tf.train.latest_checkpoint('west_checkpoints')
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="轩辕")
print(samp)

轩辕，故王夫人女为子弟。

　　太史公曰：孔氏之言“有道之方，今其有道者，有所谓之所以以为人也。”于是曰：“古者之先人之有德，其人有不有，不能用其德，则有所用之。”

　　其明年，秦伐齐，至于晋陵。十五年，楚败楚，遂取楚，取三河。二十六年，秦伐楚。三年，秦围我邯郸。二十一年，秦伐我，虏楚王。十四年，与魏攻韩而取魏。四年，秦拔我阳城，取三晋。二十一年，与楚攻赵，拔之。二十四年，秦拔我魏城。二十一年，秦攻我阳城。十六年，秦拔我城阳。二十二年，秦攻魏，拔我城城。二年，秦拔我三城，斩首三十三县。三十一年，秦拔我城阳二十一。三十四年，秦拔我城阳。十五年，秦拔我魏，拔我三城。二十六年，秦拔我三年。

　　四年，与秦战。三十六年，与魏王会临晋。十四年，伐魏，虏楚将于芒卯，十五年，取我中城。三十一年，秦拔我三城，斩首三千，秦攻我城。二十一年，与秦战，虏魏王。赵、赵、魏、魏、魏、魏、魏、赵、魏、魏、韩、赵、魏、韩、魏、韩、魏、赵、魏、魏、魏为赵、韩为韩；韩、魏、魏、魏以兵为西南，攻魏，取赵、赵，取赵，取魏，取魏。魏取赵，赵、赵。十四年，秦伐我。十一年，攻魏，取河东，虏其将赵氏。十年，与魏、赵相距。十六年，秦拔我三年，取魏城。十二年，秦拔我二年。秦拔我二年。十五年，秦拔我城阳，取魏地。

　　三十四年，秦拔我三年。

　　二十六年，秦攻我邯郸，破赵。魏败我焦、曲沃。二十三年，与魏、魏、赵、魏、魏、魏、魏、魏、魏、魏、魏、魏、魏共共攻魏，拔之。四年，秦拔我城中。二十五年，秦拔我城城。十五年，魏拔我蔺。十四年，与魏、魏、韩、魏、赵、魏、魏、魏、秦共攻赵，取我城阳。四年，魏、魏、魏以韩、魏为魏。赵与魏之战，虏魏王，虏其将魏将。二十四年，与魏战，拔魏。十一年，秦拔我二城，取魏阳城。三十一年，与秦战，败我军于城下。二十六年，秦拔魏，取三城。

　　二年，攻魏，取赵。二十一年，魏攻韩、魏，取我城阳。三十四年，秦拔我城阳。二十二年，魏拔我我阳阳。二十一年，秦拔我三年。

　　十四年，秦攻我。魏王初立，秦使赵围邯郸。秦拔我二城。三年，秦伐赵，取我城。十二年，魏、魏、魏攻魏，取河上。十二年，秦取我河外。四月，秦拔我三城。二十五年，秦败我军，斩首五十五。十四年，秦攻赵，虏韩王信，攻秦，取三河。

　　二十三年，秦拔我蔺，取魏。二十一年，秦拔我二十四城。十六年，秦拔我蔺。四年，秦拔我城城。十五年，攻秦、魏，

In [38]:
checkpoint = tf.train.latest_checkpoint('west_checkpoints')
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="我")
print(samp)

我为人，以为人，无所以得。其所为者，皆以为大人也。其后有二世，而王、魏、魏之所以为秦也。

　　秦昭王以其为大夫，秦人皆与魏王共攻秦，秦兵败，遂亡走赵。

　　赵昭襄王卒，子平陵君立。

　　惠王十年，齐田单弑其君悼襄王，十二年卒也，子立四年卒。二年，秦拔我城城。二年，秦攻我，拔我三城。十四年，秦拔魏二十城。

　　二十一年，秦拔我三城。十二年，秦拔我二城。十四年，秦拔我城阳。十六年，秦败我三十六郡。四年，秦拔我阳城。二十五年，秦拔我魏、赵。十四年，秦攻魏。十二年，秦拔我三城。四年，与魏攻魏，取魏城。二年，与韩、魏、赵、魏、赵、韩、魏共攻秦，败秦师。十四年，攻魏、魏，取之。十五年，攻秦，取三河；东至河、魏，虏赵王卬。二十四年，与秦战，虏韩相与战。十一年，秦拔我阳城，取二城，取赵地。十二年，秦败我蔺。十六年，攻魏，虏魏王。赵与秦战，取之。十二年，攻赵，取魏。二十四年，攻齐、魏，取河、河。

　　十五年，与魏、韩、韩、魏共攻魏，拔其城阳。二十六年，齐败我我城阳。十九年，齐拔我三城。十二年，秦拔我三城，取我阳阳。二年，秦伐我，拔我榆次。三十六年，攻秦，破之，拔我城。二十六年，齐攻赵。十二年，秦拔我魏。

　　十一年，赵拔我魏，取三城。十一年，赵攻韩，拔之，斩首四十四万。十二年，秦拔我二十城。十二年，秦拔我城。五十六年，秦拔我二城。三十一年，秦伐我，拔之。二十一年，与秦攻韩而取魏。三十一年，秦拔魏城。三年，攻魏。秦败我于城城。十二年，秦拔我三年。十六年，秦攻我，魏拔我城。二年，秦攻我，拔之城阳。十二年，与魏伐韩，拔我陉山，取五城，斩首四万，取魏城邑。十四年，韩、魏、赵、魏、魏、魏、齐、韩、魏、魏共攻赵，魏败赵于焦、焦，取赵、济、南地、河内、东郡。二十四年，秦拔我阳城、赵。十四年，秦拔我二城。十二年，秦拔我三城。

　　十四年，魏、韩、魏、魏、齐、魏共伐魏，拔之。十二年，秦拔我二城。十四年，秦伐我，败魏城。十四年，秦拔我蔺。二十六年，秦拔我三城，虏楚将军，与秦兵，攻魏。魏将十四年，秦攻我邯郸。十五年，秦败我于城阳。十二年，秦拔我阳城。十一年，秦拔我阳城。十一年，秦拔我城阳。二十四年，秦拔我二县，取三阳、二城，取魏城。二十六年，秦拔我蔺。二十二年，秦复拔我邯郸。秦攻魏，拔之，取城城。十六年，秦攻魏，拔之。十二年，秦拔我二城，虏其将白白。赵拔我陉，取二城，斩首三万，虏秦将将军庞涓

In [42]:
checkpoint = tf.train.latest_checkpoint('west_checkpoints')
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="情")
print(samp)

情，为之应应，为人主也。

　　齐王孙季布曰：“秦之所以为天下所以不知其功者，其所欲以为也。夫人之所以为士，不能不能胜而不知也。臣闻之，其所以不敢以言其为计也，然而不敢以为秦，不可得也。夫韩、魏之兵不能得齐，则魏必有秦也。今秦必有赵之地，秦以东攻魏，赵、魏、赵，秦之兵也。今楚、赵以攻魏，则韩必取魏之地，以秦之彊，以魏、魏之兵而攻秦也，不足以以为秦也。”赵王曰：“秦不敢为赵，秦以为韩、魏而攻韩、魏，而秦之兵不能得也，以魏之彊国之彊，而秦之所以不为为秦也，秦以彊齐、魏之兵，以为韩、赵之彊也；赵不如为赵，赵必以韩、魏之兵而攻齐，而赵之攻秦也，必以兵为赵，魏必为赵矣。秦以为秦，秦必攻韩，魏不可以为赵也。今秦之攻韩，则魏之地可与赵也；魏、魏、魏、赵、秦、赵、赵、魏之兵皆以攻魏，魏必败之，赵以为魏之兵以为韩、韩之利，必有秦秦。秦之彊秦、魏、魏、魏、魏、魏、秦、魏、赵、楚之利，必为韩、魏也。夫韩、魏之彊而齐弱而攻秦也。今秦以为秦兵，不如以韩、魏、魏、楚，秦、魏之所以为赵而不能攻赵也，而齐、魏、赵为赵，赵以为赵，魏必攻赵。秦不可与秦攻魏也。今魏以韩之彊国之利也，以为秦必危矣。故不可。秦以攻赵，魏必以兵攻秦，则秦之彊国也，不如魏之地，而齐、赵、赵之兵也，不可以为王也。且楚之所以不如魏，则魏以为魏。秦以兵攻韩、赵而拔之地，则赵以为韩，韩之地不可得而楚也。秦以为赵之彊，以为秦以为魏，秦必以韩、魏、魏，以为韩、魏之地而与韩之地也。今楚以秦之彊秦，则秦之所以为齐也，而国无以事其王也；而韩、魏之地也，而齐之彊也，而韩、魏之所以为之以不得之地也，而国之大也，以为之不可也。夫韩、魏而而秦之兵，而韩、魏不能以齐国之兵也，不可胜数而不得以兵之彊也。夫齐之地而不足以自为也，以为天下无所以以为为之所为，而国不敢为，不可。今秦必为王。臣闻秦之所以不能为秦者，而国之兵也。今大王之计，不能为王也。”于是秦使王翦以兵距韩魏。魏使秦攻楚，取其城阳，取其地。

　　二十三年，秦拔我阳城。十一年，秦拔我二城。十二年，秦拔我二年。二十五年，与秦战于城。十一年，魏拔我城阳，取我阳城，斩首十四。十二年，秦取我阳城。四年，秦拔我阳城。十二年，秦攻魏，取赵。二十四年，秦拔我城阳、五十二年。二十五年，攻赵，取河、魏，破其城阳。三十六年，与韩、魏、魏、魏与赵、魏、韩、魏、魏、韩、魏、韩、赵、魏、魏、魏、魏、魏、魏、魏、魏共伐韩，取魏

In [43]:
checkpoint = tf.train.latest_checkpoint('west_checkpoints')
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="赵")
print(samp)

赵氏，不敢以兵为国。

　　齐王之初，王子之言，以为人主，然其事也。其时，以此为相，以为天下笑。故其国有所不可者。以其为大夫，以为人所以为人也。以其故为大臣。其大夫有所与其谋，而其国无所以不为也。

　　“于是大王之行，不可以为国。夫天子之德，而国之彊，不能以为不可。夫王不听，则秦之不可以以其故不敢以为大臣也。故以其先君为之，不敢为大王。”秦王乃使人以求告之，使王曰：“王之以为秦也，必为秦王也，臣必不用其事，则王以此不敢以为王也；秦之所以为国，而秦以事秦而彊秦。今以秦之攻齐，不能以秦秦之兵，不可胜计也。今秦王之彊，则楚地之彊也。夫秦以秦攻韩之地而攻之，则秦、魏、魏、韩、魏以为魏之兵，必有韩之利也。今秦以为楚、赵之兵以为秦之利也，则楚之不得以为秦。秦、魏不能为赵，而赵必不可。今王必为魏秦之彊，不能与秦之攻也。且秦以秦彊而齐弱魏，秦以魏之攻秦，必以秦攻魏。魏不可以为魏。赵不可得矣。秦必为韩、魏之兵。今楚王以韩、魏之兵，不如赵也。”魏王以为相，为之大臣，秦王之患秦也。秦使秦之攻韩、魏，而魏以魏之为秦也，秦不可以为魏，秦必有兵以伐魏，必以韩、魏以和。秦以秦攻韩，赵必取之，魏不得以魏之地以为魏，秦之不得为地也。且王不能与魏、魏、魏、赵、魏、魏之兵皆有赵，不敢攻秦。魏必不听秦，秦之所攻而不可，必为魏。魏以魏以故为秦。”魏王曰：“善。”遂与韩、魏、魏与魏、秦、魏、魏、魏、赵、魏共攻秦。秦使韩、魏、魏、魏、魏、秦、秦、魏、魏、秦、齐、魏、赵、赵、赵、韩、魏、魏共、赵、魏、魏以魏、魏、赵、魏、魏共为韩，为魏，韩为魏。魏、魏与赵、魏、魏、魏、魏、赵、赵、赵共攻魏，取魏阳。十二年，魏败我邯郸。十五年，魏拔我蔺。四年，秦拔我阳城。十二年，秦拔我魏、梁。

　　六年，伐韩于河，至临河。十一年，秦伐魏，取我城。二十四年，秦拔我蔺。二年，秦拔我三年。二十二年，秦拔我二城。二十二年，秦攻我，拔三河、东阳。二十五年，魏攻魏，取三十城。十一年，秦取魏城，取河东。十二年，攻魏，取魏。魏、魏、魏、魏、魏为魏、魏，取魏、济。三十三年，秦败我蔺，虏其将十一城。十一年，秦败我，将十五万人，虏楚王，虏秦王。

　　三十四年，楚攻秦，拔之。十二年，攻魏，拔之，取二县。二十六年，与魏攻我阳阳，取之。十二年，与魏攻秦，拔之。十四年，秦复取韩、魏之地，复攻赵，取二十六城。十四年，魏拔魏三十城。二十一年，赵复拔我城。十一年

In [45]:
checkpoint = tf.train.latest_checkpoint('west_checkpoints')
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="故")
print(samp)

故，以为大夫。秦王以为大将军，从大将军，从大将军从大将军从击匈奴所杀，以为大王。

　　吴王将军之，以兵击匈奴之，匈奴使将军大将军为右丞相，将军青为大将军，击匈奴、单于、骠骑将军，将军大将军为大将军，从将军将骑千余人。大将军出代，击匈奴军，斩首虏二十二级，以捕斩首虏万五人。还击破匈奴军于匈奴、左将军、大将军，将军将军大将军，大将军出击匈奴大将军，军军。

　　大将军青青，大将军青，骠骑将军出朔方，以大将军为将军，从将军为将军，击匈奴、右将军出击胡，斩首虏万一级，斩首五万，以大将军军将军。大将军从骠骑将军击匈奴军大将军，将军军，大将军将军军出将军，将屯，为校尉，从大将军青，从击破军骠骑将军，斩首虏二千一级，封为大将军，封将骑将军，从击军匈奴军，斩首虏十二，以校尉。击匈奴军，大将军出，大将军从击匈奴，大将军出入定，以校尉出将军，将军击匈奴将军，斩首虏八万万。

　　校尉为骠骑将军，将军大将军出朔方，以校尉从骠骑将军为将军；出北军。至长平，封青为骠骑将军，将千骑，从骠骑将军从从大将军击匈奴，将军青，为右将军。大将军出定襄，将军李广、校尉从击匈奴将骑从骠骑将军出定襄，降汉，将军大将军，将兵将万五万骑，出定襄，匈奴将军军，将军为右将军，军军，击匈奴将军大将军，军出击匈奴大破之。军将军李沮，为左将军，军大将军，军数百，军数十万，将军将兵击破胡骑，破奴骑将军其将，斩首虏十四级，封爵大将军，将兵击匈奴大将。将军出定襄，将军骠骑将军出击匈奴，军大破之，将军屯，为骠骑将军，将军为右将军。骠骑将军从骠骑将军获将军以为功，将五千人。封大将军青，为将军，将将军击匈奴，将军李沮、李沮、将军将兵，将五千人击军大将军。匈奴将兵击匈奴大将军，斩首虏万五千人。大将军青将军，将军青、青将军击骠骑将军、大将军将，将军出击匈奴大将军，斩首虏二十二级，捕虏十五人，斩首五万，斩首十二级，赐爵一级，为大将军，将将军将军从大将军从大将军出击匈奴之马。其后匈奴大将兵击匈奴单于入汉，斩首虏万二千级。其后匈奴大将军从击单于将军军，军大将军，出陇西、北地，为右将军，击匈奴大将军，将千余人，为大司马。以校尉从大将军从骠骑将军出击匈奴，大将军出出，攻将军青，破之，将军骑骑数千骑。

　　将军青，大破之，斩首五千级，封为将军。将军将军青为校尉。青将将将军从将军将军屯北，军数万，将军军屯，斩首十八级。

　　将军青将军李沮，将军青